# Proyect_02c

### Introduction 
___
_Objective:_ __To explore, segment and group the neighbourhoods of the city of Toronto.__   
The information is obtained with web scraping by Wikipedia and the data is transformed into a structured data format.  

### Table of contents 

#### Notebook: 3 / 3
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#ref1">Import Data</a></li>
        <li><a href="#ref2">Notebook 1: Data Wrangling</a></li>
        <li><a href="#ref3">Notebook 2: Geolocation</a></li>
        <li><a href="#ref4">Notebook 3: GeoData Exploration</a></li>
        <li><a href="#ref5">Notebook 3: Resources</a></li>
        <li><a href="#ref5">Notebook 3: Conclusion</a></li>
    </ol>
</div>
<br>

<a id="ref1"></a>
##  Import Data
***

In [1]:
!pip install beautifulsoup4 # HTML and XML data extraction library.
!pip install request # requests , timeout
!pip install folium # map rendering library
!pip install geopy # convert an address into latitude and longitude values

Notebook 1: Data Wrangling    
https://github.com/Azhura/Coursera_Capstone/blob/master/Project02a.ipynb

In [2]:
# Notebook: 1 / 3
# Importing Packages
import pandas as pd # DataFrame
import numpy as np # Arrays
from bs4 import BeautifulSoup # HTML and XML data extraction library.
import requests # requests , timeout
# Loading the url data.
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page_response = requests.get(url,timeout=5) # requests , timeout
soup = BeautifulSoup(page_response.content,'lxml') # Transforming to BeautifulSoup object
table = soup.find_all('table')[0] # Filtering the html data table
df = pd.read_html(str(table))[0] # Transforming data with pandas
# Value Filtering
na = df['Borough'] != 'Not assigned' 
df_na = df[na] 
new_data = df_na
new_data['Neighborhood'] = new_data['Neighborhood'].str.replace('/',',')
new_data  =  new_data.sort_values (['Postal code','Borough','Neighborhood'])
new_data.index = np.arange(0, len(new_data)) # changing start index
print("Size: ",new_data.shape)
new_data.head() # Head After

Size:  (103, 3)


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Notebook 2: Geolocation   
https://github.com/Azhura/Coursera_Capstone/blob/master/Project02b.ipynb

In [3]:
# IBM Cloud Code -> Read - Geospatial_Coordinates.csv
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_01ba54beae4e4c2ba72338f0bffbcfc0 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='WHQpda1UemxiKh1dWr6ktIildCxStzrXvpilUjPCWLCL',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_01ba54beae4e4c2ba72338f0bffbcfc0.get_object(Bucket='datascience-donotdelete-pr-rxrupxu6gwy6yg',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_data_0 = pd.read_csv(body)
#------------------------------End -> IBM Cloud Code-------------------------------
# Joining data frames by index.
new_data02 = new_data.join(df_data_0, lsuffix='index', rsuffix='index')
# Removing the repeated column
new_data02.drop(['Postal Code'],axis=1,inplace=True)
print("Load Complete: new_data02 = Geospatial_Coordinates.csv")
print("size: ",new_data02.shape)
new_data02.head()

Load Complete: new_data02 = Geospatial_Coordinates.csv
size:  (103, 5)


,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<a id="ref4"></a>
# GeoData Exploration

In [4]:
new_data02.groupby('Borough').count()

,Postal code,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Scarborough,17,17,17,17
West Toronto,6,6,6,6


In [5]:
print('There are {} uniques categories.'.format(len(new_data02['Borough'].unique())))

There are 10 uniques categories.


In [6]:
Central_Toronto = new_data02[(new_data02['Borough']=='Central Toronto')]
Downtown_Toronto = new_data02[(new_data02['Borough']=='Downtown Toronto')]
East_Toronto = new_data02[(new_data02['Borough']=='East Toronto')]
West_Toronto = new_data02[(new_data02['Borough']=='West Toronto')]

In [7]:
frames = [Central_Toronto, Downtown_Toronto, East_Toronto,West_Toronto ]
result = pd.concat(frames)
result.index = np.arange(0, len(result)) # changing index
print("Size: ",result.shape)
result.head()

Size:  (39, 5)


,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park , Summerhill East",43.689574,-79.383160


In [8]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(result['Borough'].unique()),
        result.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


### Import library 

In [9]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm 
import matplotlib.colors as colors
from sklearn.cluster import KMeans # import k-means from clustering stage
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
print('Imported Data')

Imported Data


### Transforming a direction into latitude and longitude.
* Looking for Toronto in Canada.

In [10]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Creating a toronto map
* With a few neighborhood.

In [11]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, postalcode, borough, neighborhood in zip(result['Latitude'], 
                                                       result['Longitude'],
                                                       result['Postal code'], 
                                                       result['Borough'], 
                                                       result['Neighborhood']):
    label = '{}, {}'.format(neighborhood, postalcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### Grouping the west of toronto 

In [12]:
West_Toronto_data = result[result['Borough'] == 'West Toronto'].reset_index(drop=True)
West_Toronto_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M6H,West Toronto,"Dufferin , Dovercourt Village",43.669005,-79.442259
1,M6J,West Toronto,"Little Portugal , Trinity",43.647927,-79.419750
2,M6K,West Toronto,"Brockton , Parkdale Village , Exhibition Place",43.636847,-79.428191
3,M6P,West Toronto,"High Park , The Junction South",43.661608,-79.464763
4,M6R,West Toronto,"Parkdale , Roncesvalles",43.648960,-79.456325


In [13]:
West_Toronto_data.loc[0, 'Neighborhood']

'Dufferin , Dovercourt Village'

In [14]:
neighborhood_latitude = West_Toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = West_Toronto_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = West_Toronto_data.loc[0, 'Neighborhood'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Dufferin , Dovercourt Village are 43.66900510000001, -79.4422593.


In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(West_Toronto_data['Latitude'], West_Toronto_data['Longitude'], West_Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Defining the credentials and version of the Foursquare

In [16]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'KHOZKSCNCIFW1UAJHPFETKYEH5W0GFBA4RCZPKXVD45VMXEU' # your Foursquare ID
CLIENT_SECRET = 'C2WQDAOLLVZYVTBW5XEEAT2ASUT3XFNDQMPRGDR0P2SUXGZQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KHOZKSCNCIFW1UAJHPFETKYEH5W0GFBA4RCZPKXVD45VMXEU
CLIENT_SECRET:C2WQDAOLLVZYVTBW5XEEAT2ASUT3XFNDQMPRGDR0P2SUXGZQ


### Let's explore the first neighborhood in our dataframe.

In [17]:
# West_Toronto_data.loc[0, 'Neighborhood']
neighborhood_latitude = West_Toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = West_Toronto_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = West_Toronto_data.loc[0, 'Neighborhood'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Dufferin , Dovercourt Village are 43.66900510000001, -79.4422593.


* Now, let's get the top 100 venues that are in Dufferin and Dovercourt Village within a radius of 500 meters.

In [18]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=KHOZKSCNCIFW1UAJHPFETKYEH5W0GFBA4RCZPKXVD45VMXEU&client_secret=C2WQDAOLLVZYVTBW5XEEAT2ASUT3XFNDQMPRGDR0P2SUXGZQ&v=20180605&ll=43.66900510000001,-79.4422593&radius=500&limit=100'

* Send the GET request and examine the resutls

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e97cf2d71c428001b9eb0e1'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Davenport',
  'headerFullLocation': 'Davenport, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 17,
  'suggestedBounds': {'ne': {'lat': 43.67350510450001,
    'lng': -79.43604977526607},
   'sw': {'lat': 43.664505095500004, 'lng': -79.44846882473394}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5753753b498eeb535c53aed5',
       'name': 'The Greater Good Bar',
       'location': {'address': '229 Geary St',
        'crossStreet': 'at Dufferin St',
        'lat': 43.669409,
        'lng': -79.439267,
        'labeledLatLngs': [{'label': 'disp

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

* Now we are ready to clean the json and structure it into a pandas dataframe.

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print("size: ",nearby_venues.shape)
nearby_venues

size:  (17, 4)


,name,categories,lat,lng
0,The Greater Good Bar,Bar,43.669409,-79.439267
1,Parallel,Middle Eastern Restaurant,43.669516,-79.438728
2,Planet Fitness,Gym / Fitness Center,43.667588,-79.442574
3,Blood Brothers Brewing,Brewery,43.669944,-79.436533
4,FreshCo,Grocery Store,43.667918,-79.440754
5,Happy Bakery & Pastries,Bakery,43.667050,-79.441791
6,Rehearsal Factory,Music Venue,43.668877,-79.443603
7,Nova Era Bakery,Bakery,43.669886,-79.437582
8,The Sovereign,Café,43.673116,-79.440265
9,TD Canada Trust,Bank,43.667934,-79.441698


In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

17 venues were returned by Foursquare.


In [23]:
# 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
toronto_venues = getNearbyVenues(names=West_Toronto_data['Neighborhood'],
                                   latitudes=West_Toronto_data['Latitude'],
                                   longitudes=West_Toronto_data['Longitude']
                                  )

Dufferin , Dovercourt Village
Little Portugal , Trinity
Brockton , Parkdale Village , Exhibition Place
High Park , The Junction South
Parkdale , Roncesvalles
Runnymede , Swansea


In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(156, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Dufferin , Dovercourt Village",43.669005,-79.442259,The Greater Good Bar,43.669409,-79.439267,Bar
1,"Dufferin , Dovercourt Village",43.669005,-79.442259,Parallel,43.669516,-79.438728,Middle Eastern Restaurant
2,"Dufferin , Dovercourt Village",43.669005,-79.442259,Planet Fitness,43.667588,-79.442574,Gym / Fitness Center
3,"Dufferin , Dovercourt Village",43.669005,-79.442259,Blood Brothers Brewing,43.669944,-79.436533,Brewery
4,"Dufferin , Dovercourt Village",43.669005,-79.442259,FreshCo,43.667918,-79.440754,Grocery Store


In [26]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Brockton , Parkdale Village , Exhibition Place",22,22,22,22,22,22
"Dufferin , Dovercourt Village",17,17,17,17,17,17
"High Park , The Junction South",24,24,24,24,24,24
"Little Portugal , Trinity",43,43,43,43,43,43
"Parkdale , Roncesvalles",14,14,14,14,14,14
"Runnymede , Swansea",36,36,36,36,36,36


In [27]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 79 uniques categories.


## Analyze Each Neighborhood

In [28]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.tail()

,Neighborhood,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Beer Store,Bistro,...,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
151,"Runnymede , Swansea",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
152,"Runnymede , Swansea",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
153,"Runnymede , Swansea",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
154,"Runnymede , Swansea",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
155,"Runnymede , Swansea",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
toronto_onehot.shape

(156, 80)

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Beer Store,Bistro,...,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.045455,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Dufferin , Dovercourt Village",0.000000,0.058824,0.000000,0.000000,0.117647,0.058824,0.058824,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000
2,"High Park , The Junction South",0.041667,0.000000,0.041667,0.000000,0.041667,0.000000,0.083333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Little Portugal , Trinity",0.000000,0.023256,0.000000,0.046512,0.000000,0.000000,0.116279,0.023256,0.023256,...,0.000000,0.023256,0.000000,0.000000,0.023256,0.046512,0.023256,0.023256,0.000000,0.023256
4,"Parkdale , Roncesvalles",0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.071429,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Runnymede , Swansea",0.000000,0.000000,0.000000,0.000000,0.000000,0.027778,0.027778,0.000000,0.000000,...,0.055556,0.000000,0.027778,0.000000,0.000000,0.027778,0.000000,0.000000,0.000000,0.000000


In [31]:
toronto_grouped.shape

(6, 80)

In [32]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Brockton , Parkdale Village , Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3             Gym  0.05
4    Climbing Gym  0.05


----Dufferin , Dovercourt Village----
           venue  freq
0       Pharmacy  0.12
1         Bakery  0.12
2           Café  0.06
3  Grocery Store  0.06
4    Music Venue  0.06


----High Park , The Junction South----
                venue  freq
0                 Bar  0.08
1  Mexican Restaurant  0.08
2     Thai Restaurant  0.08
3                Café  0.08
4        Antique Shop  0.04


----Little Portugal , Trinity----
         venue  freq
0          Bar  0.12
1   Restaurant  0.07
2  Coffee Shop  0.05
3         Café  0.05
4  Men's Store  0.05


----Parkdale , Roncesvalles----
                         venue  freq
0                    Gift Shop  0.14
1                   Restaurant  0.07
2               Breakfast Spot  0.07
3                Movie Theater  0.07
4  Eastern European Restaurant  0

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Breakfast Spot,Gym,Restaurant,Grocery Store,Intersection,Italian Restaurant,Convenience Store,Nightclub
1,"Dufferin , Dovercourt Village",Bakery,Pharmacy,Park,Brewery,Grocery Store,Wine Shop,Gym / Fitness Center,Middle Eastern Restaurant,Music Venue,Café
2,"High Park , The Junction South",Café,Thai Restaurant,Bar,Mexican Restaurant,Antique Shop,Furniture / Home Store,Grocery Store,Fried Chicken Joint,Flea Market,Italian Restaurant
3,"Little Portugal , Trinity",Bar,Restaurant,Café,Vegetarian / Vegan Restaurant,Men's Store,Coffee Shop,Asian Restaurant,Miscellaneous Shop,Cocktail Bar,Korean Restaurant
4,"Parkdale , Roncesvalles",Gift Shop,Dessert Shop,Dog Run,Cuban Restaurant,Eastern European Restaurant,Bank,Bar,Coffee Shop,Italian Restaurant,Bookstore


## Cluster Neighborhoods

In [35]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 2, 4, 1, 0, 1], dtype=int32)

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = West_Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M6H,West Toronto,"Dufferin , Dovercourt Village",43.669005,-79.442259,2,Bakery,Pharmacy,Park,Brewery,Grocery Store,Wine Shop,Gym / Fitness Center,Middle Eastern Restaurant,Music Venue,Café
1,M6J,West Toronto,"Little Portugal , Trinity",43.647927,-79.419750,1,Bar,Restaurant,Café,Vegetarian / Vegan Restaurant,Men's Store,Coffee Shop,Asian Restaurant,Miscellaneous Shop,Cocktail Bar,Korean Restaurant
2,M6K,West Toronto,"Brockton , Parkdale Village , Exhibition Place",43.636847,-79.428191,3,Café,Coffee Shop,Breakfast Spot,Gym,Restaurant,Grocery Store,Intersection,Italian Restaurant,Convenience Store,Nightclub
3,M6P,West Toronto,"High Park , The Junction South",43.661608,-79.464763,4,Café,Thai Restaurant,Bar,Mexican Restaurant,Antique Shop,Furniture / Home Store,Grocery Store,Fried Chicken Joint,Flea Market,Italian Restaurant
4,M6R,West Toronto,"Parkdale , Roncesvalles",43.648960,-79.456325,0,Gift Shop,Dessert Shop,Dog Run,Cuban Restaurant,Eastern European Restaurant,Bank,Bar,Coffee Shop,Italian Restaurant,Bookstore


In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster 1

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,West Toronto,0,Gift Shop,Dessert Shop,Dog Run,Cuban Restaurant,Eastern European Restaurant,Bank,Bar,Coffee Shop,Italian Restaurant,Bookstore


#### Cluster 2

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,West Toronto,1,Bar,Restaurant,Café,Vegetarian / Vegan Restaurant,Men's Store,Coffee Shop,Asian Restaurant,Miscellaneous Shop,Cocktail Bar,Korean Restaurant
5,West Toronto,1,Café,Coffee Shop,Sushi Restaurant,Pizza Place,Pub,Italian Restaurant,Fish & Chips Shop,Falafel Restaurant,Indie Movie Theater,Grocery Store


#### Cluster 3

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,West Toronto,2,Bakery,Pharmacy,Park,Brewery,Grocery Store,Wine Shop,Gym / Fitness Center,Middle Eastern Restaurant,Music Venue,Café


#### Cluster 4

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,West Toronto,3,Café,Coffee Shop,Breakfast Spot,Gym,Restaurant,Grocery Store,Intersection,Italian Restaurant,Convenience Store,Nightclub


#### Cluster 5

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,West Toronto,4,Café,Thai Restaurant,Bar,Mexican Restaurant,Antique Shop,Furniture / Home Store,Grocery Store,Fried Chicken Joint,Flea Market,Italian Restaurant


This notebook was created by [Carlos Alberto Gómez Prado](https://www.linkedin.com/in/carlospradobigdata/), as an assignment for the IBM coursera course.   
This notebook is part of a course on Coursera called Applied Data Science Capstone. 

---